# 05wk-021: 취업+밸런스게임, 오버피팅

최규빈  
2023-10-05

# 1. 강의영상

# 2. Imports

In [22]:
import numpy as np
import pandas as pd
import sklearn.linear_model

# 3. 밸런스게임

`-` ref: <https://verovero1.tistory.com/136>

`-` 하고 싶은 말: 내가 원한다면 무수히 많은 데이터를 모을 수 있다고
가정하자.[1]

# 4. 상황1

`-` 아래의 자료를 고려하자.

[1] 빅데이터?????

In [4]:
data = {
    'toeic': [640, 705, 930, 755, 410, 655, 400, 915, 970, 895],
    'gpa': [0.712533, 0.639031, 0.148400, 1.230271, 3.279419, 1.255110, 4.157389, 3.714847, 1.584432, 2.170776],
    'employment': [1, 0, 0, 0, 1, 0, 1, 1, 1, 0],
    'balance_game_1': [1, 0, 0, 0, 1, 0, 1, 0, 0, 0]
}

df = pd.DataFrame(data)
df

`-` train/test를 아래와 같이 구분했다고 하자.

In [6]:
df_train = df[:7]
df_train

In [7]:
df_test = df[7:]
df_test

`-` 직감: 이 자료를 가지고 학습하면 반드시 망한다..

`-` 결론: 자료가 많을수록 좋은게 아니다. 쓸모없는 자료가 있다면 오히려
학습에 방해가 된다. 위의 자료로 학습한다면 오버피팅이 발생할 것.

`-` 반론: 예제가 너무 억지 아니야?

# 5. 상황2

`-` 다시 아래의 자료를 가정하자.

In [63]:
data = {
    'toeic': [640, 705, 930, 755, 410, 655, 400, 915, 970, 895],
    'gpa': [0.712533, 0.639031, 0.148400, 1.230271, 3.279419, 1.255110, 4.157389, 3.714847, 1.584432, 2.170776],
    'employment': [1, 0, 0, 0, 1, 0, 1, 1, 1, 0]
}
df = pd.DataFrame(data)
df

`-` 밸런스게임을 50번정도 진행했다고 하자.

In [64]:
np.random.seed(43052)
arr = (np.random.rand(10*50).reshape(10,50)>0.5) * 1
df_balance= pd.DataFrame(arr,columns=[f'X{i}' for i in range(50)])
df = pd.concat([df,df_balance],axis=1)
df

In [87]:
(df.X12 + df.X43 + df.X8 - df.X29) > 0 

In [88]:
df.employment

`-` 수틀리면 한 50억번 진행할 수도 있어..

# 6. 이상한 자료로 분석

`-` 취업자료를 변경

In [107]:
df = pd.read_csv('https://raw.githubusercontent.com/guebin/MP2023/main/posts/employment.csv')
df

In [108]:
df_balance = pd.DataFrame((np.random.randn(500,5000)>0.5).reshape(500,5000)*1,columns = ['X'+str(i) for i in range(5000)])
df_merged = pd.concat([df,df_balance],axis=1)
df_merged

`-` (X,y), (XX,yy)

In [115]:
X = df_merged.drop('employment',axis=1)[:400]
y = df_merged[['employment']][:400]
XX = df_merged.drop('employment',axis=1)[400:]
yy = df_merged[['employment']][400:]

`-` predictor 생성, 학습, 평가

In [121]:
prdtr = sklearn.linear_model.LogisticRegression()
prdtr.fit(X,y)

/home/cgb2/anaconda3/envs/ag/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/cgb2/anaconda3/envs/ag/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(

In [122]:
print(f'train score: {prdtr.score(X,y)}')
print(f'test score: {prdtr.score(XX,yy)}')

train score: 1.0
test score: 0.76

# 7. 올바른 자료로 분석

`-` 취업자료

In [123]:
df = pd.read_csv('https://raw.githubusercontent.com/guebin/MP2023/main/posts/employment.csv')
df

`-` (X,y), (XX,yy)

In [124]:
X = df.drop('employment',axis=1)[:400]
y = df[['employment']][:400]
XX = df.drop('employment',axis=1)[400:]
yy = df[['employment']][400:]

`-` predictor 생성, 학습, 평가

In [125]:
prdtr = sklearn.linear_model.LogisticRegression()
prdtr.fit(X,y)

/home/cgb2/anaconda3/envs/ag/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)

In [126]:
print(f'train score: {prdtr.score(X,y)}')
print(f'test score: {prdtr.score(XX,yy)}')

train score: 0.8925
test score: 0.83

# 8. 오버피팅

## A. 학부1~2학년 수준의 설명

`-` 오버피팅: 머신러닝과 통계에서 자주 나타나는 문제로, 모델이 학습
데이터에 과도하게 최적화되어 실제로 새로운 데이터나 테스트 데이터에서의
성능이 저하되는 현상을 말함.

`-` 오버피팅의 원인:

-   불필요한 특징: 불필요한 특징이 데이터에 포함되어 있을 때 오버피팅이
    발생할 수 있음.
-   데이터 부족: 사용 가능한 학습 데이터가 매우 적을 때 모델은 해당
    데이터에만 특화된 패턴을 학습할 가능성이 있음.

## B. 일반인 수준의 설명

`-` 시험 공부(1): 공부를 하랬더니 외우고 있음..

`-` 시험 공부(2): (시험 하루 전날에) 공부 그만하고 술이나 먹으러 가자..
더 공부하면 train error만 줄일 뿐이야..

`-` 운전: 특정도로에서만 운전연습을 했음. 그래서 그 도로의 구멍, 곡률,
신호등의 위치까지 완벽하게 숙지하였음. 그 결과 그 도로에서는 잘하게
되었지만, 그 도로 이외의 다른도로에서 운전을 한다면 문제가 발생함.

`-` 언어: 특정 주제나 특정 상황에 대한 대화만을 반복적으로 연습하여, 그
상황에서는 완벽한 대화가 가능하지만 그 외의 상황에서는 대화를 제대로
이어나갈 수 없음.

## C. 내 개념